In [ ]:
#upload images
# הורדת קובץ ה-ZIP מגוגל דרייב
#https://drive.google.com/file/d/1-329XUOi4XaagdlutAqvPi4yKB-bX9U9/view?usp=drive_link
zip_id = "1-329XUOi4XaagdlutAqvPi4yKB-bX9U9"
!gdown --id {zip_id} -O food_10.zip

# חילוץ הקבצים
!unzip food_10.zip -d food_10

In [ ]:
# Preprocessing and Ingest Data
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Activation, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


In [ ]:
#use ImageDataGenerator to create train_datagen and test_datagen
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
#create train/test generators, make sure to use class_mode='categorical'
#use taret_size 200x200
train_generator = train_datagen.flow_from_directory(
    'food_10/train/',
    target_size=(200, 200),
    batch_size=32,
    class_mode='categorical',
    seed=42
)

test_generator = test_datagen.flow_from_directory(
    'food_10/test/',
    target_size=(200, 200),
    batch_size=32,
    class_mode='categorical',
    seed=42
)


In [ ]:
#build a model, use at leasrt one BatchNormalization layer and at least on Dropout layer
#please make sure the SHAPE OF THE OUTPUT LAYER IS CORRECTLY DEFINED
# Build Model
model = Sequential([
    Conv2D(10, (3, 3), input_shape=(200, 200, 3)),
    BatchNormalization(),
    Activation(activation='relu'),
    Dropout(0.2),
    Conv2D(10, 3, activation='relu'),
    MaxPool2D(),
    Dropout(0.2),
    Conv2D(10, 3, activation='relu'),
    MaxPool2D(),
    Dropout(0.2),
    Flatten(),
    Dense(32, activation ='relu'),
    Dense(10, activation='softmax')
])

# Compile & Fit Model with RMSprop
model.compile(optimizer=RMSprop(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_generator,
    epochs=10,#mach less even 3..
    validation_data=test_generator

)


In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(len(history.history['loss'])), history.history['loss'], label="training_loss")
plt.plot(range(len(history.history['loss'])), history.history['val_loss'], label="val_loss")
plt.legend()


In [ ]:
# Test the model
img_path ='/content/food_10/test/pancakes/101644.jpg'#pancake
img = load_img(img_path, target_size=(200, 200))  #גודל תואם אימון

# convert the image with numpy array with img_to_array method
img_array = img_to_array(img)

#use np.expAnd_dims to add 1 more dim. on axis=0 to the array
img_array = np.expand_dims(img_array, axis=0)

# normalize the result
img_array = img_array / 255.0

#predict
prediction=model.predict(img_array)
selected_label = np.argmax(prediction)


In [ ]:
# שליפת שם הקטגוריה ישירות מהמילון
class_labels = {v: k for k, v in train_generator.class_indices.items()}# הופכים את המילון
#{v: k for k, v in train_generator.class_indices.items()}
predicted_class = class_labels[selected_label]  # שליפת שם הקטגוריה לפי מספר

# הצגת התוצאה
print(f"Predicted category: {predicted_class}")

# הצגת התמונה
plt.imshow(img)
plt.axis("off")
plt.show()